In [1]:
import pandas as pd
import numpy as np
from P6_code.FinishedCode.importData import ImportEV, ImportWeather
from P6_code.FinishedCode.dataTransformation import createUsers
import plotly.express as px
import plotly.graph_objects as go

In [2]:
start, end = "2018-05-01", "2018-11-01"

In [15]:
weather = ImportWeather().getPasadena(agg='hour')
weather.rename(columns={'Hour (PST)' : 'Hour'}, inplace=True)
weather['Hour'] = weather['Hour']/100 -1
#weather = weather.loc[(weather.index.date >= start) & (weather.index.date < end)]
weather.drop(['Jul', 'ETo (mm)', 'Sol Rad (W/sq.m)', 'Vap Pres (kPa)', 'Rel Hum (%)', 'Dew Point (C)', 'Wind Dir (0-360)', 'Soil Temp (C)'], axis='columns', inplace=True)
weather

,Date,Hour,Precip (mm),Air Temp (C),Wind Speed (m/s)
0,2018-02-02,0.0,0.0,12.7,1.2
1,2018-02-02,1.0,0.0,12.0,1.3
2,2018-02-02,2.0,0.0,12.0,1.4
3,2018-02-02,3.0,0.0,11.7,1.3
4,2018-02-02,4.0,0.0,11.7,1.2
...,...,...,...,...,...
7987,2018-12-31,19.0,0.0,12.9,4.7
7988,2018-12-31,20.0,0.0,12.8,4.6
7989,2018-12-31,21.0,0.0,12.4,3.2
7990,2018-12-31,22.0,0.0,11.8,2.9


In [4]:
ev_data = ImportEV().getCaltech(start_date=start, end_date=end, removeUsers=False, userSampleLimit=25)
Users = createUsers(ev_data, start, end).data
#Users['dailyPrecip'] = [weather['Precip (mm)'][i] for i in Users['connectionTime'].dt.strftime('%Y-%m-%d')]
#Users['dailyTemp'] = [weather['Air Temp (C)'][i] for i in Users['connectionTime'].dt.strftime('%Y-%m-%d')]
#Users['dailyWind'] = [weather['Wind Speed (m/s)'][i] for i in Users['connectionTime'].dt.strftime('%Y-%m-%d')]
Users['chargingTime'] = (pd.to_datetime(Users["doneChargingTime"]) - pd.to_datetime(Users["connectionTime"])) / np.timedelta64(1, 's')
Users.loc[:, ("Date")] = pd.to_datetime(Users.connectionTime.dt.strftime('%Y-%m-%d'))
Users.loc[:, ("Hour")] = Users.connectionTime.dt.hour.astype(float)

#User_61 = Users.getUserData(user="000000061")
#User_61['Date'] = pd.to_datetime([f"{User_61['Year'][i]}-{User_61['Month'][i]}-{User_61['Day'][i]}" for i in User_61.index])

Users

,_id,clusterID,connectionTime,disconnectTime,doneChargingTime,kWhDelivered,sessionID,siteID,spaceID,stationID,timezone,userID,userInputs,chargingTime,Date,Hour
270,5bc9160af9af8b0dad3c0690,0039,2018-05-01 06:23:18+00:00,2018-05-01 18:53:41+00:00,2018-05-01 10:17:07+00:00,13.410000,2_39_95_27_2018-05-01 13:23:18.374684,0002,CA-319,2-39-95-27,America/Los_Angeles,None,None,14029.0,2018-05-01,6.0
271,5bc9160af9af8b0dad3c0691,0039,2018-05-01 06:54:34+00:00,2018-05-01 16:15:35+00:00,2018-05-01 08:35:39+00:00,8.618000,2_39_79_380_2018-05-01 13:54:33.599590,0002,CA-489,2-39-79-380,America/Los_Angeles,None,None,6065.0,2018-05-01,6.0
272,5bc9160af9af8b0dad3c0692,0039,2018-05-01 07:46:04+00:00,2018-05-01 17:54:07+00:00,2018-05-01 16:57:02+00:00,13.969000,2_39_139_28_2018-05-01 14:46:04.441389,0002,CA-303,2-39-139-28,America/Los_Angeles,None,None,33058.0,2018-05-01,7.0
273,5bc9160af9af8b0dad3c0693,0039,2018-05-01 07:56:29+00:00,2018-05-01 14:52:45+00:00,2018-05-01 09:07:56+00:00,6.914000,2_39_79_378_2018-05-01 14:56:28.908623,0002,CA-326,2-39-79-378,America/Los_Angeles,None,None,4287.0,2018-05-01,7.0
274,5bc9160af9af8b0dad3c0694,0039,2018-05-01 08:14:50+00:00,2018-05-01 15:51:30+00:00,2018-05-01 13:18:12+00:00,13.785000,2_39_88_24_2018-05-01 15:14:50.443137,0002,CA-314,2-39-88-24,America/Los_Angeles,None,None,18202.0,2018-05-01,8.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13108,5c2e8550f9af8b13dab074d9,0039,2018-10-31 21:53:58+00:00,2018-10-31 22:32:43+00:00,2018-10-31 22:33:42+00:00,3.372000,2_39_90_26_2018-11-01 04:53:58.292201,0002,CA-316,2-39-90-26,America/Los_Angeles,None,None,2384.0,2018-10-31,21.0
13109,5c2e8550f9af8b13dab074da,0039,2018-10-31 22:32:47+00:00,2018-10-31 22:53:40+00:00,2018-10-31 22:53:00+00:00,1.389000,2_39_90_26_2018-11-01 05:32:47.339653,0002,CA-316,2-39-90-26,America/Los_Angeles,None,None,1213.0,2018-10-31,22.0
13110,5c2e8550f9af8b13dab074db,0039,2018-10-31 22:40:49+00:00,2018-11-01 00:57:28+00:00,2018-11-01 00:27:00+00:00,1.225597,2_39_78_365_2018-11-01 05:40:48.678916,0002,CA-321,2-39-78-365,America/Los_Angeles,None,None,6371.0,2018-10-31,22.0
13111,5c2e8550f9af8b13dab074dc,0039,2018-10-31 22:53:48+00:00,2018-10-31 23:24:48+00:00,2018-10-31 23:24:16+00:00,0.962000,2_39_90_440_2018-11-01 05:53:48.467907,0002,CA-500,2-39-90-440,America/Los_Angeles,None,None,1828.0,2018-10-31,22.0


In [16]:
df = pd.merge(weather, Users, on=['Date', 'Hour'])
df

,Date,Hour,Precip (mm),Air Temp (C),Wind Speed (m/s),_id,clusterID,connectionTime,disconnectTime,doneChargingTime,kWhDelivered,sessionID,siteID,spaceID,stationID,timezone,userID,userInputs,chargingTime
0,2018-05-01,6.0,0.1,10.9,1.2,5bc9160af9af8b0dad3c0690,0039,2018-05-01 06:23:18+00:00,2018-05-01 18:53:41+00:00,2018-05-01 10:17:07+00:00,13.410000,2_39_95_27_2018-05-01 13:23:18.374684,0002,CA-319,2-39-95-27,America/Los_Angeles,None,None,14029.0
1,2018-05-01,6.0,0.1,10.9,1.2,5bc9160af9af8b0dad3c0691,0039,2018-05-01 06:54:34+00:00,2018-05-01 16:15:35+00:00,2018-05-01 08:35:39+00:00,8.618000,2_39_79_380_2018-05-01 13:54:33.599590,0002,CA-489,2-39-79-380,America/Los_Angeles,None,None,6065.0
2,2018-05-01,7.0,0.0,11.2,1.1,5bc9160af9af8b0dad3c0692,0039,2018-05-01 07:46:04+00:00,2018-05-01 17:54:07+00:00,2018-05-01 16:57:02+00:00,13.969000,2_39_139_28_2018-05-01 14:46:04.441389,0002,CA-303,2-39-139-28,America/Los_Angeles,None,None,33058.0
3,2018-05-01,7.0,0.0,11.2,1.1,5bc9160af9af8b0dad3c0693,0039,2018-05-01 07:56:29+00:00,2018-05-01 14:52:45+00:00,2018-05-01 09:07:56+00:00,6.914000,2_39_79_378_2018-05-01 14:56:28.908623,0002,CA-326,2-39-79-378,America/Los_Angeles,None,None,4287.0
4,2018-05-01,8.0,0.0,11.7,1.3,5bc9160af9af8b0dad3c0694,0039,2018-05-01 08:14:50+00:00,2018-05-01 15:51:30+00:00,2018-05-01 13:18:12+00:00,13.785000,2_39_88_24_2018-05-01 15:14:50.443137,0002,CA-314,2-39-88-24,America/Los_Angeles,None,None,18202.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12838,2018-10-31,21.0,0.0,15.3,1.1,5c2e8550f9af8b13dab074d9,0039,2018-10-31 21:53:58+00:00,2018-10-31 22:32:43+00:00,2018-10-31 22:33:42+00:00,3.372000,2_39_90_26_2018-11-01 04:53:58.292201,0002,CA-316,2-39-90-26,America/Los_Angeles,None,None,2384.0
12839,2018-10-31,22.0,0.0,14.4,1.2,5c2e8550f9af8b13dab074da,0039,2018-10-31 22:32:47+00:00,2018-10-31 22:53:40+00:00,2018-10-31 22:53:00+00:00,1.389000,2_39_90_26_2018-11-01 05:32:47.339653,0002,CA-316,2-39-90-26,America/Los_Angeles,None,None,1213.0
12840,2018-10-31,22.0,0.0,14.4,1.2,5c2e8550f9af8b13dab074db,0039,2018-10-31 22:40:49+00:00,2018-11-01 00:57:28+00:00,2018-11-01 00:27:00+00:00,1.225597,2_39_78_365_2018-11-01 05:40:48.678916,0002,CA-321,2-39-78-365,America/Los_Angeles,None,None,6371.0
12841,2018-10-31,22.0,0.0,14.4,1.2,5c2e8550f9af8b13dab074dc,0039,2018-10-31 22:53:48+00:00,2018-10-31 23:24:48+00:00,2018-10-31 23:24:16+00:00,0.962000,2_39_90_440_2018-11-01 05:53:48.467907,0002,CA-500,2-39-90-440,America/Los_Angeles,None,None,1828.0


In [17]:
df.corr()

,Hour,Precip (mm),Air Temp (C),Wind Speed (m/s),kWhDelivered,chargingTime
Hour,1.000000,-0.020271,0.156022,0.080018,-0.008798,-0.192444
Precip (mm),-0.020271,1.000000,-0.103839,-0.012659,0.014448,0.030761
Air Temp (C),0.156022,-0.103839,1.000000,0.056726,-0.045926,-0.082434
Wind Speed (m/s),0.080018,-0.012659,0.056726,1.000000,-0.002874,-0.030622
kWhDelivered,-0.008798,0.014448,-0.045926,-0.002874,1.000000,0.542397
chargingTime,-0.192444,0.030761,-0.082434,-0.030622,0.542397,1.000000


In [18]:
px.imshow(df.corr(), color_continuous_scale='RdBu', color_continuous_midpoint=0)